In [1]:
import pandas as pd
from tqdm import tqdm

from dash_website import DIMENSIONS, MAIN_CATEGORIES_TO_CATEGORIES


list_indexes = []
for dimension in DIMENSIONS + ["All_aging_dimensions"]:
    for category in MAIN_CATEGORIES_TO_CATEGORIES["All"] + [
        f"All_{main_category}" for main_category in MAIN_CATEGORIES_TO_CATEGORIES.keys()
    ]:
        list_indexes.append([dimension, category])
indexes = pd.MultiIndex.from_tuples(list_indexes, names=["dimension", "category"])

list_columns = []
for item in ["total", "significant", "accelerated_aging", "decelerated_aging"]:
    if item == "total":
        observations = ["total"]
    else:
        observations = ["number", "percentage"]
    for observation in observations:
        list_columns.append([item, observation])
columns = pd.MultiIndex.from_tuples(list_columns, names=["item", "observation"])

summary = pd.DataFrame(None, index=indexes, columns=columns)
summary

item                                        total significant             \
observation                                 total      number percentage   
dimension            category                                              
Abdomen              Alcohol                  NaN         NaN        NaN   
                     AnthropometryBodySize    NaN         NaN        NaN   
                     AnthropometryImpedance   NaN         NaN        NaN   
                     ArterialStiffness        NaN         NaN        NaN   
                     BloodBiochemistry        NaN         NaN        NaN   
...                                           ...         ...        ...   
All_aging_dimensions All_Biomarkers           NaN         NaN        NaN   
                     All_Phenotypes           NaN         NaN        NaN   
                     All_Diseases             NaN         NaN        NaN   
                     All_Environmental        NaN         NaN        NaN   
                     All_Socioeconomics       NaN         NaN        NaN   

item                                        accelerated_aging             \
observation                                            number percentage   
dimension            category                                              
Abdomen              Alcohol                              NaN        NaN   
                     AnthropometryBodySize                NaN        NaN   
                     AnthropometryImpedance               NaN        NaN   
                     ArterialStiffness                    NaN        NaN   
                     BloodBiochemistry                    NaN        NaN   
...                                                       ...        ...   
All_aging_dimensions All_Biomarkers                       NaN        NaN   
                     All_Phenotypes                       NaN        NaN   
                     All_Diseases                         NaN        NaN   
                     All_Environmental                    NaN        NaN   
                     All_Socioeconomics                   NaN        NaN   

item                                        decelerated_aging             
observation                                            number percentage  
dimension            category                                             
Abdomen              Alcohol                              NaN        NaN  
                     AnthropometryBodySize                NaN        NaN  
                     AnthropometryImpedance               NaN        NaN  
                     ArterialStiffness                    NaN        NaN  
                     BloodBiochemistry                    NaN        NaN  
...                                                       ...        ...  
All_aging_dimensions All_Biomarkers                       NaN        NaN  
                     All_Phenotypes                       NaN        NaN  
                     All_Diseases                         NaN        NaN  
                     All_Environmental                    NaN        NaN  
                     All_Socioeconomics                   NaN        NaN  

[3060 rows x 7 columns]

In [54]:
import pandas as pd
from tqdm import tqdm

from dash_website.utils.aws_loader import load_feather
from dash_website import DIMENSIONS, MAIN_CATEGORIES_TO_CATEGORIES


if __name__ == "__main__":
    list_indexes = []
    for dimension in DIMENSIONS + ["All_aging_dimensions"]:
        for category in MAIN_CATEGORIES_TO_CATEGORIES["All"] + [
            f"All_{main_category}" for main_category in MAIN_CATEGORIES_TO_CATEGORIES.keys()
        ]:
            list_indexes.append([dimension, category])
    indexes = pd.MultiIndex.from_tuples(list_indexes, names=["dimension", "category"])

    list_columns = []
    for item in ["total", "significant", "accelerated_aging", "decelerated_aging"]:
        if item == "total":
            observations = ["total"]
        else:
            observations = ["number", "percentage"]
        for observation in observations:
            list_columns.append([item, observation])
    columns = pd.MultiIndex.from_tuples(list_columns, names=["item", "observation"])

    summary = pd.DataFrame(None, index=indexes, columns=columns)

    for dimension in tqdm(DIMENSIONS):
        correlations_dimension = load_feather(
            f"xwas/univariate_results/linear_correlations_{dimension}.feather",
            columns=["category", "p_value", "correlation"],
        )

        for group in correlations_dimension.groupby(by=["category"]):
            category = group[0]
            summary.loc[(dimension, category), ("total", "total")] = group[1].shape[0]
            summary.loc[(dimension, category), ("significant", "number")] = (
                group[1]["p_value"] < 0.05 / group[1].shape[0]
            ).sum()
            summary.loc[(dimension, category), ("accelerated_aging", "number")] = (
                (group[1]["p_value"] < 0.05 / group[1].shape[0]) & (group[1]["correlation"] < 0)
            ).sum()
            summary.loc[(dimension, category), ("decelerated_aging", "number")] = (
                (group[1]["p_value"] < 0.05 / group[1].shape[0]) & (group[1]["correlation"] > 0)
            ).sum()

        for main_category, categories in MAIN_CATEGORIES_TO_CATEGORIES.items():
            group = correlations_dimension.set_index("category").loc[categories]

            summary.loc[(dimension, f"All_{main_category}"), ("total", "total")] = group.shape[0]
            summary.loc[(dimension, f"All_{main_category}"), ("significant", "number")] = (
                group["p_value"] < 0.05 / group.shape[0]
            ).sum()
            summary.loc[(dimension, f"All_{main_category}"), ("accelerated_aging", "number")] = (
                (group["p_value"] < 0.05 / group.shape[0]) & (group["correlation"] < 0)
            ).sum()
            summary.loc[(dimension, f"All_{main_category}"), ("decelerated_aging", "number")] = (
                (group["p_value"] < 0.05 / group.shape[0]) & (group["correlation"] > 0)
            ).sum()

        for item in ["significant", "accelerated_aging", "decelerated_aging"]:
            summary.loc[dimension, (item, "percentage")] = (
                summary.loc[dimension, (item, "number")] / summary.loc[dimension, ("total", "total")]
            ).values

    # summary.columns = map(str, summary.columns.tolist())
    # summary.reset_index().to_feather("data/xwas/univariate_results/summary.feather")

ImportError: cannot import name 'ssl' from 'urllib3.util.ssl_' (c:\Dossier\HMS\Dash-Website\env_website\lib\site-packages\urllib3\util\ssl_.py)

In [22]:
import pandas as pd 

summary_load = pd.read_feather("../../data/xwas/univariate_results/summary.feather").set_index(["dimension", "category"])
summary_load.columns = pd.MultiIndex.from_tuples(
    list(map(eval, summary_load.columns.tolist())), names=["item", "observation"]
)

summary.loc[:, :] = summary_load
summary_load

item                                   total significant             \
observation                            total      number percentage   
dimension       category                                              
Abdomen         Alcohol                   29           7   0.241379   
                AnthropometryBodySize      8           6   0.750000   
                AnthropometryImpedance     5           5   1.000000   
                ArterialStiffness          8           2   0.250000   
                BloodBiochemistry         28          14   0.500000   
...                                      ...         ...        ...   
set_instances23 All_Biomarkers          3124         179   0.057298   
                All_Phenotypes           182          10   0.054945   
                All_Diseases            2073           2   0.000965   
                All_Environmental        261          14   0.053640   
                All_Socioeconomics        85           2   0.023529   

item                                   accelerated_aging             \
observation                                       number percentage   
dimension       category                                              
Abdomen         Alcohol                                5   0.172414   
                AnthropometryBodySize                  1   0.125000   
                AnthropometryImpedance                 5   1.000000   
                ArterialStiffness                      2   0.250000   
                BloodBiochemistry                      8   0.285714   
...                                                  ...        ...   
set_instances23 All_Biomarkers                        90   0.028809   
                All_Phenotypes                         8   0.043956   
                All_Diseases                           2   0.000965   
                All_Environmental                      7   0.026820   
                All_Socioeconomics                     1   0.011765   

item                                   decelerated_aging             
observation                                       number percentage  
dimension       category                                             
Abdomen         Alcohol                                2   0.068966  
                AnthropometryBodySize                  5   0.625000  
                AnthropometryImpedance                 0   0.000000  
                ArterialStiffness                      0   0.000000  
                BloodBiochemistry                      6   0.214286  
...                                                  ...        ...  
set_instances23 All_Biomarkers                        89   0.028489  
                All_Phenotypes                         2   0.010989  
                All_Diseases                           0   0.000000  
                All_Environmental                      7   0.026820  
                All_Socioeconomics                     1   0.011765  

[2970 rows x 7 columns]

In [18]:
column_without_percentage = [("total", "total"), ("significant", "number"), ("accelerated_aging", "number"), ("decelerated_aging", "number")]

In [28]:
category = "Alcohol"

summary.loc[("All_aging_dimensions", category), column_without_percentage] = summary_load.swaplevel().loc[category, column_without_percentage].sum()

summary.loc["All_aging_dimensions"]

for item in ["significant", "accelerated_aging", "decelerated_aging"]:
    summary.loc["All_aging_dimensions", (item, "percentage")] = (
        summary.loc["All_aging_dimensions", (item, "number")] / summary.loc["All_aging_dimensions"
, ("total", "total")]
    ).values
summary.loc["All_aging_dimensions"]

item                    total significant            accelerated_aging  \
observation             total      number percentage            number   
category                                                                 
Alcohol                 957.0       336.0   0.351097             219.0   
AnthropometryBodySize     NaN         NaN        NaN               NaN   
AnthropometryImpedance    NaN         NaN        NaN               NaN   
ArterialStiffness         NaN         NaN        NaN               NaN   
BloodBiochemistry         NaN         NaN        NaN               NaN   
...                       ...         ...        ...               ...   
All_Biomarkers            NaN         NaN        NaN               NaN   
All_Phenotypes            NaN         NaN        NaN               NaN   
All_Diseases              NaN         NaN        NaN               NaN   
All_Environmental         NaN         NaN        NaN               NaN   
All_Socioeconomics        NaN         NaN        NaN               NaN   

item                              decelerated_aging             
observation            percentage            number percentage  
category                                                        
Alcohol                   0.22884             117.0   0.122257  
AnthropometryBodySize         NaN               NaN        NaN  
AnthropometryImpedance        NaN               NaN        NaN  
ArterialStiffness             NaN               NaN        NaN  
BloodBiochemistry             NaN               NaN        NaN  
...                           ...               ...        ...  
All_Biomarkers                NaN               NaN        NaN  
All_Phenotypes                NaN               NaN        NaN  
All_Diseases                  NaN               NaN        NaN  
All_Environmental             NaN               NaN        NaN  
All_Socioeconomics            NaN               NaN        NaN  

[90 rows x 7 columns]